In [1]:
# STEP 1 — Pull American League standings for today

import requests
from datetime import date

today = date.today().isoformat()

standings_url = "https://statsapi.mlb.com/api/v1/standings"
params = {
    "leagueId": 103,    # 103 = AL, 104 = NL
    "season": 2025,
    "date": today
}

resp = requests.get(standings_url, params=params)
print("HTTP status:", resp.status_code)
print("Requested URL:", resp.url)

standings = resp.json()
print("Top-level keys:", list(standings.keys()))


HTTP status: 200
Requested URL: https://statsapi.mlb.com/api/v1/standings?leagueId=103&season=2025&date=2025-09-05
Top-level keys: ['copyright', 'records']


In [2]:
# STEP 2 — Explore standings structure

records = standings.get("records", [])
print("Number of records (divisions):", len(records))

if records:
    first_record = records[0]
    print("Record keys:", list(first_record.keys()))

    team_records = first_record.get("teamRecords", [])
    print("Number of team records:", len(team_records))

    if team_records:
        print("Team record keys:", list(team_records[0].keys()))
        print("Nested 'team' keys:", list(team_records[0].get("team", {}).keys()))


Number of records (divisions): 3
Record keys: ['standingsType', 'league', 'division', 'sport', 'lastUpdated', 'teamRecords']
Number of team records: 5
Team record keys: ['team', 'season', 'streak', 'divisionRank', 'leagueRank', 'sportRank', 'gamesPlayed', 'gamesBack', 'wildCardGamesBack', 'leagueGamesBack', 'springLeagueGamesBack', 'sportGamesBack', 'divisionGamesBack', 'conferenceGamesBack', 'leagueRecord', 'lastUpdated', 'records', 'runsAllowed', 'runsScored', 'divisionChamp', 'divisionLeader', 'hasWildcard', 'clinched', 'eliminationNumber', 'eliminationNumberSport', 'eliminationNumberLeague', 'eliminationNumberDivision', 'eliminationNumberConference', 'wildCardEliminationNumber', 'magicNumber', 'wins', 'losses', 'runDifferential', 'winningPercentage']
Nested 'team' keys: ['id', 'name', 'link']


In [7]:
# STEP 3 — Build Silver standings table with league/division lookups

import requests
import pandas as pd

clean_rows = []

# Loop through each division record in standings
for rec in records:
    league_id = (rec.get("league") or {}).get("id")
    division_id = (rec.get("division") or {}).get("id")

    for tr in rec.get("teamRecords", []):
        team = tr.get("team", {}) or {}

        row = {
            "season": params.get("season"),
            "date": params.get("date"),
            "league_id": league_id,
            "division_id": division_id,
            "team_id": team.get("id"),
            "team_name": team.get("name"),
            "wins": tr.get("wins"),
            "losses": tr.get("losses"),
            "pct": tr.get("winningPercentage"),
            "gamesBack": tr.get("gamesBack"),
            "wildCardGamesBack": tr.get("wildCardGamesBack"),
            "divisionLeader": tr.get("divisionLeader"),
        }
        clean_rows.append(row)

df_standings = pd.DataFrame(clean_rows)

# --- Add league + division names via lookups ---
# Fetch league lookup
leagues = requests.get("https://statsapi.mlb.com/api/v1/leagues").json()["leagues"]
df_leagues = pd.DataFrame([{"league_id": l["id"], "league": l["name"]} for l in leagues])

# Fetch division lookup
divs = requests.get("https://statsapi.mlb.com/api/v1/divisions").json()["divisions"]
df_divs = pd.DataFrame([{"division_id": d["id"], "division": d["name"]} for d in divs])

# Join lookups into standings
df_standings = df_standings.merge(df_leagues, on="league_id", how="left")
df_standings = df_standings.merge(df_divs, on="division_id", how="left")

print("Shape:", df_standings.shape)
df_standings.head(10)


Shape: (15, 14)


,season,date,league_id,division_id,team_id,team_name,wins,losses,pct,gamesBack,wildCardGamesBack,divisionLeader,league,division
0,2025,2025-09-05,103,201,141,Toronto Blue Jays,81,59,.579,-,-,True,American League,American League East
1,2025,2025-09-05,103,201,147,New York Yankees,78,62,.557,3.0,+5.0,False,American League,American League East
2,2025,2025-09-05,103,201,111,Boston Red Sox,78,63,.553,3.5,+4.5,False,American League,American League East
3,2025,2025-09-05,103,201,139,Tampa Bay Rays,71,69,.507,10.0,2.0,False,American League,American League East
4,2025,2025-09-05,103,201,110,Baltimore Orioles,64,76,.457,17.0,9.0,False,American League,American League East
5,2025,2025-09-05,103,202,116,Detroit Tigers,81,60,.574,-,-,True,American League,American League Central
6,2025,2025-09-05,103,202,118,Kansas City Royals,71,69,.507,9.5,2.0,False,American League,American League Central
7,2025,2025-09-05,103,202,114,Cleveland Guardians,69,70,.496,11.0,3.5,False,American League,American League Central
8,2025,2025-09-05,103,202,142,Minnesota Twins,62,78,.443,18.5,11.0,False,American League,American League Central
9,2025,2025-09-05,103,202,145,Chicago White Sox,53,88,.376,28.0,20.5,False,American League,American League Central
